## Normal Augmentations+

This notebook provides the code for training a network with the following augmentation settings: <br>
* Geometric Augmentations (e.g., Horizontal Flip, Rotation, etc.)
* Photometric Augmentations (e.g., Autocontrast, Equalization)
* Downsampling
* Common Corruptions
* Amplitude-Phase Recombination
<br><br>

Note: 
- This code is heavily based on the Amplitude-Phase Recombination github.

References:
* Amplitude-Phase Recombination: https://github.com/iCGY96/APR
* Common Corruptions: https://github.com/bethgelab/imagecorruptions

    

### Import Libraries

In [1]:
## import libraries

import os
import sys
import argparse
import datetime
import time
import csv
import os.path as osp
import numpy as np
import warnings
import importlib
import pandas as pd
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import torchvision
from datasets import CIFAR10D, CIFAR100D, CustomDataset
from utils.utils import AverageMeter, Logger, save_networks, load_networks
from core import train, test, test_robustness, test_two_datasets

parser = argparse.ArgumentParser("Training")


### Model Settings and Parameters

In [2]:
parser = argparse.ArgumentParser("Training")
options = {}

# dataset
options['data'] = './data'
options['outf'] = './results'
options['dataset'] = 'mosquito'  ## USUALLY MODIFIED: 'cifar10' or 'mosquito'
options['dataset_mosq'] = 'HQ100' ## when mosquito dataset is selected, choose dataset ratio ('HQ100': High Quality 100%, 'HQ100LQ20': HQ 100% and LQ 20%)
options['workers'] = 8 ## number of data loading workers (default: 2)
options['input_size'] = 224 ## USUALLY MODIFIED

# optimization
options['batch_size'] = 32 ## USUALLY MODIFIED
options['lr'] = 0.1 ## model learning rate
options['max_epoch'] = 200
options['stepsize'] = 30
options['aug'] = 'aprs' ## USUALLY MODIFIED: ['none', 'aprs', 'new-aug', 'amp-adj']

# model
options['model'] = 'resnet18' ## ['resnet18', 'wide_resnet', allconv, 'densenet', 'resnext']

# misc
options['eval_freq'] = 10
options['print_freq'] = 100
options['gpu'] = '0'
options['seed'] = 0
options['use_cpu'] = True
options['eval'] = False ## train or evaluate

# parameters for generating adversarial examples
options['epsilon'] = 0.0157 ## maximum perturbation of adversaries (4/255=0.0157)
options['alpha'] = 0.00784 ## movement multiplier per iteration when generating adversarial examples (2/255=0.00784)
options['k'] = 10 ## maximum iteration when generating adversarial examples
options['perturbation_type'] = 'linf' ## the type of the perturbation ('linf' or 'l2')


if not os.path.exists(options['outf']):
    os.makedirs(options['outf'])

if not os.path.exists(options['data']):
    os.makedirs(options['data'])

# misc 2
options['outf'] = "None" ## USUALLY MODIFIED: checkpoint address ["./results/checkpoints/NAMEOFEXPERIMENT/", "None"]
options['actual_print'] = 4  ## number of actual print frequency (i.e., the number of loss values shown per epoch)


if options['outf'] == "None":
    options['outf'] = "./results/checkpoints/DefaultBin/"


### Set the seed and GPU

In [3]:
## Set the seed and use GPU when available unless explicitly set to CPU in the options above
torch.manual_seed(options['seed'])
os.environ['CUDA_VISIBLE_DEVICES'] = options['gpu']
use_gpu = torch.cuda.is_available()
if options['use_cpu']: use_gpu = False

options.update({'use_gpu': use_gpu})

if use_gpu:
    print("Currently using GPU: {}".format(options['gpu']))
    cudnn.benchmark = True
    torch.cuda.manual_seed_all(options['seed'])
else:
    print("Currently using CPU")

Currently using CPU


### Set-up the dataset to use

In [5]:
if options['dataset'] == 'cifar10':
    Data = CIFAR10D(dataroot=options['data'], batch_size=options['batch_size'], _transforms=options['aug'], _eval=options['eval'])
    OODData = CIFAR100D(dataroot=options['data'], batch_size=options['batch_size'], _transforms=options['aug'])
    trainloader, testloader, outloader = Data.train_loader, Data.test_loader, OODData.test_loader

elif options['dataset'] == 'mosquito': ## for mosquito dataset
    if options['dataset_mosq'] == 'HQ100':
        data_dir = {
            'train':'R:/Datasets/mosquito/sets/raw-high/train/',   ## using train folder
            'test':'R:/Datasets/mosquito/sets/raw-high/test/',     ## using test folder
            'eval':'R:/Datasets/mosquito/raw-hl/low/',
        }

    elif options['dataset_mosq'] == 'HQ100LQ20':
        data_dir = {
            'train':'R:/Datasets/mosquito/sets/raw-comb/100-20/train/',   ## using combined train folder (HQ100% - LQ20%)
            'test':'R:/Datasets/mosquito/sets/raw-high/test/',     ## using test folder
            'eval':'R:/Datasets/mosquito/sets/raw-low/test/',

        }
    Data = CustomDataset(dataroot=data_dir, batch_size=options['batch_size'], _transforms=options['aug'], _eval=True, input_size=options['input_size'])

    ## Initialize the dataloader
    trainloader, testloader, outloader = Data.train_loader, Data.test_loader, Data.out_loaders

else: ## for CIFAR100 dataset
    Data = CIFAR100D(dataroot=options['data'], batch_size=options['batch_size'], _transforms=options['aug'], _eval=options['eval'])
    OODData = CIFAR10D(dataroot=options['data'], batch_size=options['batch_size'], _transforms=options['aug'])
    trainloader, testloader, outloader = Data.train_loader, Data.test_loader, OODData.test_loader


num_classes = Data.num_classes

## modify the print frequency based on the trainloader
options['print_freq'] = int(len(trainloader)/(options['actual_print']))

print("Creating model: {}".format(options['model']))
if 'wide_resnet' in options['model']:
    print('wide_resnet')
    from model.wide_resnet import WideResNet
    net = WideResNet(40, num_classes, 2, 0.0)
elif 'allconv' in options['model']:
    print('allconv')
    from model.allconv import AllConvNet
    net = AllConvNet(num_classes)
elif 'densenet' in options['model']:
    print('densenet')
    from model.densenet import  densenet
    net = densenet(num_classes=num_classes)
elif 'resnext' in options['model']:
    print('resnext29')
    from model.resnext import resnext29
    net = resnext29(num_classes)
else:
    print('resnet18')
    from model.resnet import ResNet18
    net = ResNet18(num_classes=num_classes)

# define loss function (criterion) and optimizer
criterion = nn.CrossEntropyLoss().cuda()

if use_gpu:
    net = nn.DataParallel(net, device_ids=[i for i in range(len(options['gpu'].split(',')))]).cuda()
    criterion = criterion.cuda()

file_name = '{}_{}_{}'.format(options['model'], options['dataset'], options['aug'])

## reload last saved network
#net.load_state_dict(torch.load("results/checkpoints/3_27_exp13/wider_resnet_28_10_mosquito_amp-adj_.pth"))
#criterion.load_state_dict(torch.load("results/checkpoints/3_27_exp13/wider_resnet_28_10_mosquito_amp-adj__criterion.pth"))

if options['eval']:
    net, criterion = load_networks(net, options['outf'], file_name, criterion=criterion)
    outloaders = Data.out_loaders
    results = test(net, criterion, testloader, outloader, epoch=0, **options)
    acc = results['ACC']
    res = dict()
    res['ACC'] = dict()
    acc_res = []
    for key in Data.out_keys:
        results = test_robustness(net, criterion, outloaders[key], epoch=0, label=key, **options)
        print('{} (%): {:.3f}\t'.format(key, results['ACC']))
        res['ACC'][key] = results['ACC']
        acc_res.append(results['ACC'])
    print('Mean ACC:', np.mean(acc_res))
    print('Mean Error:', 100-np.mean(acc_res))



params_list = [{'params': net.parameters()},
            {'params': criterion.parameters()}]


optimizer = torch.optim.SGD(params_list, lr=options['lr'], momentum=0.9, nesterov=True, weight_decay=5e-4)
scheduler = lr_scheduler.MultiStepLR(optimizer, gamma=0.2, milestones=[60, 120, 160, 190])

start_time = time.time()

best_acc = 0.0
for epoch in range(options['max_epoch']):
    print("==> Epoch {}/{}".format(epoch+1, options['max_epoch']))

    train(net, criterion, optimizer, trainloader, epoch=epoch, **options)

    if options['eval_freq'] > 0 and (epoch+1) % options['eval_freq'] == 0 or (epoch+1) == options['max_epoch'] or epoch > 200:
        print("==> Test")
        results = test_two_datasets(net, criterion, testloader, outloader, epoch=epoch, **options)

        if best_acc < results['ACC']:
            best_acc = results['ACC']
            print("Best Acc (%): {:.3f}\t".format(best_acc))
        
        save_networks(net, options['outf'], file_name, criterion=criterion)

    scheduler.step()

elapsed = round(time.time() - start_time)
elapsed = str(datetime.timedelta(seconds=elapsed))
print("Finished. Total elapsed time (h:m:s): {}".format(elapsed))



APRecombination aprs
Creating model: resnet18
resnet18
==> Epoch 1/200


RuntimeError: indices should be either on cpu or on the same device as the indexed tensor (cpu)